In [4]:
import pandas as pd
import numpy as np
import json
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model

# Load intents from the JSON file
with open("intents.json", "r") as file:
    intents_data = json.load(file)["intents"]

# Extract patterns and tags from intents
patterns = []
tags = []
for intent in intents_data:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        tags.append(intent["tag"])

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
patterns_tfidf = vectorizer.fit_transform(patterns).toarray()

# Create LabelEncoder for tags
le = LabelEncoder()
tags_encoded = le.fit_transform(tags)

# Define and compile the model
model = Sequential([
    Dense(30, input_shape=(len(patterns_tfidf[0]),)),
    Dense(28),
    Dense(28),
    Dense(26),
    Dense(26),
    Dense(24),
    Dense(len(set(tags)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(patterns_tfidf, tags_encoded, epochs=50, batch_size=14)

# Save the trained model
model.save("hospital_bot_model")

print("Model saved successfully")

Epoch 1/50
4/4 [==============================] - 4s 13ms/step - loss: 2.2000 - accuracy: 0.0851
Epoch 2/50
4/4 [==============================] - 0s 12ms/step - loss: 2.1434 - accuracy: 0.2340
Epoch 3/50
4/4 [==============================] - 0s 10ms/step - loss: 2.0934 - accuracy: 0.2979
Epoch 4/50
4/4 [==============================] - 0s 17ms/step - loss: 2.0436 - accuracy: 0.4255
Epoch 5/50
4/4 [==============================] - 0s 13ms/step - loss: 1.9906 - accuracy: 0.4468
Epoch 6/50
4/4 [==============================] - 0s 10ms/step - loss: 1.9276 - accuracy: 0.5106
Epoch 7/50
4/4 [==============================] - 0s 17ms/step - loss: 1.8588 - accuracy: 0.5745
Epoch 8/50
4/4 [==============================] - 0s 13ms/step - loss: 1.7725 - accuracy: 0.6383
Epoch 9/50
4/4 [==============================] - 0s 10ms/step - loss: 1.6788 - accuracy: 0.6596
Epoch 10/50
4/4 [==============================] - 0s 10ms/step - loss: 1.5725 - accuracy: 0.6809
Epoch 11/50
4/4 [============

In [ ]:
def start_chat():
    print("---------------  AI Chat bot  ---------------")
    print("Ask any queries...")
    print("I will try to understand you and reply...")
    print("Type EXIT to quit...")
    while True:
        inp = input("Ask anything... : ")
        if inp == "EXIT":
            break
        else:
            if inp:
                # Preprocess input using the loaded vectorizer
                inp_data_tfidf = vectorizer.transform([inp.lower()]).toarray()
                # Predict intent tag
                predicted_proba = model.predict(inp_data_tfidf)
                encoded_label = np.argmax(predicted_proba)
                predicted_tag = le.inverse_transform([encoded_label])[0]
                # Get a random response for the predicted tag
                responses = [intent["responses"] for intent in intents_data if intent["tag"] == predicted_tag]
                response = random.choice(responses[0]) if responses else "I'm sorry, I don't understand."
                print("Response... : ", response)
            else:
                pass

# Start chatting
start_chat()

---------------  AI Chat bot  ---------------
Ask any queries...
I will try to understand you and reply...
Type EXIT to quit...
Ask anything... : Hello!
1/1 [==============================] - 0s 115ms/step
Response... :  Good to see you again
Ask anything... : Thank you!
1/1 [==============================] - 0s 33ms/step
Response... :  Any time!
Ask anything... : How could you help me ?
1/1 [==============================] - 0s 22ms/step
Response... :  I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies
Ask anything... : Please provide hospital type
1/1 [==============================] - 0s 23ms/step
Response... :  Please provide hospital name or location
Ask anything... : Cluj
1/1 [==============================] - 0s 23ms/step
Response... :  Good to see you again
Ask anything... : Please provide hospital type
1/1 [==============================] - 0s 22ms/step
Response... :  Please provide hospital name or location
Ask anything... : 

In [10]:
import json
import random
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load intents from the JSON file
with open("intents.json", "r") as file:
    intents_data = json.load(file)["intents"]

# Extract patterns and tags from intents
patterns = []
tags = []
for intent in intents_data:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        tags.append(intent["tag"])

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
patterns_tfidf = vectorizer.fit_transform(patterns).toarray()

# Create LabelEncoder for tags
le = LabelEncoder()
tags_encoded = le.fit_transform(tags)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(patterns_tfidf, tags_encoded, test_size=0.2, random_state=42)

# Define and compile the model
model = Sequential([
    Dense(30, input_shape=(len(patterns_tfidf[0]),)),
    Dense(28),
    Dense(28),
    Dense(26),
    Dense(26),
    Dense(len(set(tags)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=14)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Generate predictions on the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Decode labels
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred_labels)

# Print classification report
print(classification_report(y_test_labels, y_pred_labels))

Epoch 1/50
3/3 [==============================] - 1s 7ms/step - loss: 2.1961 - accuracy: 0.0541
Epoch 2/50
3/3 [==============================] - 0s 7ms/step - loss: 2.1619 - accuracy: 0.2162
Epoch 3/50
3/3 [==============================] - 0s 7ms/step - loss: 2.1316 - accuracy: 0.2703
Epoch 4/50
3/3 [==============================] - 0s 6ms/step - loss: 2.1029 - accuracy: 0.2973
Epoch 5/50
3/3 [==============================] - 0s 6ms/step - loss: 2.0734 - accuracy: 0.3784
Epoch 6/50
3/3 [==============================] - 0s 6ms/step - loss: 2.0440 - accuracy: 0.4324
Epoch 7/50
3/3 [==============================] - 0s 6ms/step - loss: 2.0146 - accuracy: 0.5676
Epoch 8/50
3/3 [==============================] - 0s 6ms/step - loss: 1.9848 - accuracy: 0.6486
Epoch 9/50
3/3 [==============================] - 0s 6ms/step - loss: 1.9517 - accuracy: 0.7027
Epoch 10/50
3/3 [==============================] - 0s 6ms/step - loss: 1.9199 - accuracy: 0.7027
Epoch 11/50
3/3 [======================

1/1 [==============================] - 0s 333ms/step - loss: 1.7946 - accuracy: 0.3000
Test Loss: 1.7945531606674194
Test Accuracy: 0.30000001192092896


1/1 [==============================] - 0s 316ms/step
                       precision    recall  f1-score   support

         adverse_drug       0.00      0.00      0.00         3
       blood_pressure       0.00      0.00      0.00         1
blood_pressure_search       0.00      0.00      0.00         1
             greeting       0.14      1.00      0.25         1
      hospital_search       0.00      0.00      0.00         1
              options       1.00      1.00      1.00         1
      pharmacy_search       0.00      0.00      0.00         1
               thanks       1.00      1.00      1.00         1

             accuracy                           0.30        10
            macro avg       0.27      0.38      0.28        10
         weighted avg       0.21      0.30      0.23        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
import json
import random
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load intents from the JSON file
with open("intents.json", "r") as file:
    intents_data = json.load(file)["intents"]

# Extract patterns and tags from intents
patterns = []
tags = []
for intent in intents_data:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        tags.append(intent["tag"])

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
patterns_tfidf = vectorizer.fit_transform(patterns).toarray()

# Create LabelEncoder for tags
le = LabelEncoder()
tags_encoded = le.fit_transform(tags)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(patterns_tfidf, tags_encoded, test_size=0.2, random_state=42)

# Define and compile the model
model = Sequential([
    Dense(60, input_shape=(len(patterns_tfidf[0]),), activation='relu'),
    Dense(50),
    Dense(40),
    Dense(26),
    Dense(26),
    Dense(24),
    Dense(len(set(tags)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=14)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Generate predictions on the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Decode labels
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred_labels)

# Print classification report
print(classification_report(y_test_labels, y_pred_labels))

Epoch 1/50
3/3 [==============================] - 4s 11ms/step - loss: 2.2285 - accuracy: 0.1622
Epoch 2/50
3/3 [==============================] - 0s 11ms/step - loss: 2.1626 - accuracy: 0.2162
Epoch 3/50
3/3 [==============================] - 0s 28ms/step - loss: 2.1145 - accuracy: 0.2703
Epoch 4/50
3/3 [==============================] - 0s 11ms/step - loss: 2.0662 - accuracy: 0.2432
Epoch 5/50
3/3 [==============================] - 0s 10ms/step - loss: 2.0153 - accuracy: 0.3514
Epoch 6/50
3/3 [==============================] - 0s 13ms/step - loss: 1.9602 - accuracy: 0.4865
Epoch 7/50
3/3 [==============================] - 0s 10ms/step - loss: 1.8933 - accuracy: 0.5946
Epoch 8/50
3/3 [==============================] - 0s 10ms/step - loss: 1.8176 - accuracy: 0.6216
Epoch 9/50
3/3 [==============================] - 0s 10ms/step - loss: 1.7322 - accuracy: 0.6757
Epoch 10/50
3/3 [==============================] - 0s 12ms/step - loss: 1.6318 - accuracy: 0.6486
Epoch 11/50
3/3 [============

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

As we can see the accuracy of our model is quite low , so we can try to improve it , the first step is improving it with increasing the complexity of the neural network:

In [16]:
import json
import random
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load intents from the JSON file
with open("intents.json", "r") as file:
    intents_data = json.load(file)["intents"]

# Extract patterns and tags from intents
patterns = []
tags = []
for intent in intents_data:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        tags.append(intent["tag"])

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
patterns_tfidf = vectorizer.fit_transform(patterns).toarray()

# Create LabelEncoder for tags
le = LabelEncoder()
tags_encoded = le.fit_transform(tags)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(patterns_tfidf, tags_encoded, test_size=0.2, random_state=42)

# Define and compile the model
model = Sequential([
    Dense(128, input_shape=(len(patterns_tfidf[0]),), activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(set(tags)), activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=14)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Generate predictions on the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Decode labels
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred_labels)

# Print classification report
print(classification_report(y_test_labels, y_pred_labels))


Epoch 1/50
3/3 [==============================] - 3s 11ms/step - loss: 2.2032 - accuracy: 0.1081
Epoch 2/50
3/3 [==============================] - 0s 10ms/step - loss: 2.1779 - accuracy: 0.2703
Epoch 3/50
3/3 [==============================] - 0s 12ms/step - loss: 2.1596 - accuracy: 0.3784
Epoch 4/50
3/3 [==============================] - 0s 13ms/step - loss: 2.1402 - accuracy: 0.6486
Epoch 5/50
3/3 [==============================] - 0s 20ms/step - loss: 2.1214 - accuracy: 0.7297
Epoch 6/50
3/3 [==============================] - 0s 11ms/step - loss: 2.0999 - accuracy: 0.7838
Epoch 7/50
3/3 [==============================] - 0s 12ms/step - loss: 2.0763 - accuracy: 0.7838
Epoch 8/50
3/3 [==============================] - 0s 15ms/step - loss: 2.0494 - accuracy: 0.7838
Epoch 9/50
3/3 [==============================] - 0s 13ms/step - loss: 2.0201 - accuracy: 0.7568
Epoch 10/50
3/3 [==============================] - 0s 10ms/step - loss: 1.9858 - accuracy: 0.7568
Epoch 11/50
3/3 [============

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


It is obvious that the model is not performing much better , so we need to try to further improve it ,this time using dropout layers to introduce regularization:

In [17]:
import json
import random
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
# Load intents from the JSON file
with open("intents.json", "r") as file:
    intents_data = json.load(file)["intents"]

# Extract patterns and tags from intents
patterns = []
tags = []
for intent in intents_data:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        tags.append(intent["tag"])

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
patterns_tfidf = vectorizer.fit_transform(patterns).toarray()

# Create LabelEncoder for tags
le = LabelEncoder()
tags_encoded = le.fit_transform(tags)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(patterns_tfidf, tags_encoded, test_size=0.2, random_state=42)

# Define and compile the model
model = Sequential([
    Dense(128, input_shape=(len(patterns_tfidf[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(len(set(tags)), activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=14)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Generate predictions on the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Decode labels
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred_labels)

# Print classification report
print(classification_report(y_test_labels, y_pred_labels))


Epoch 1/50
3/3 [==============================] - 1s 7ms/step - loss: 2.2255 - accuracy: 0.0811
Epoch 2/50
3/3 [==============================] - 0s 7ms/step - loss: 2.1726 - accuracy: 0.1622
Epoch 3/50
3/3 [==============================] - 0s 9ms/step - loss: 2.1867 - accuracy: 0.1351
Epoch 4/50
3/3 [==============================] - 0s 8ms/step - loss: 2.1617 - accuracy: 0.1892
Epoch 5/50
3/3 [==============================] - 0s 7ms/step - loss: 2.1509 - accuracy: 0.1622
Epoch 6/50
3/3 [==============================] - 0s 8ms/step - loss: 2.2014 - accuracy: 0.1892
Epoch 7/50
3/3 [==============================] - 0s 8ms/step - loss: 2.1626 - accuracy: 0.1622
Epoch 8/50
3/3 [==============================] - 0s 7ms/step - loss: 2.1589 - accuracy: 0.2432
Epoch 9/50
3/3 [==============================] - 0s 7ms/step - loss: 2.1657 - accuracy: 0.2162
Epoch 10/50
3/3 [==============================] - 0s 7ms/step - loss: 2.1652 - accuracy: 0.1622
Epoch 11/50
3/3 [======================

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


As we can see our models performance decreased , we can try to add more epochs , in order to achieve better results:

In [22]:
import json
import random
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
# Load intents from the JSON file
with open("intents.json", "r") as file:
    intents_data = json.load(file)["intents"]

# Extract patterns and tags from intents
patterns = []
tags = []
for intent in intents_data:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        tags.append(intent["tag"])

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
patterns_tfidf = vectorizer.fit_transform(patterns).toarray()

# Create LabelEncoder for tags
le = LabelEncoder()
tags_encoded = le.fit_transform(tags)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(patterns_tfidf, tags_encoded, test_size=0.2, random_state=42)

# Define and compile the model
model = Sequential([
    Dense(128, input_shape=(len(patterns_tfidf[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(len(set(tags)), activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=14)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Generate predictions on the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Decode labels
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred_labels)

# Print classification report
print(classification_report(y_test_labels, y_pred_labels))


Epoch 1/100
3/3 [==============================] - 1s 6ms/step - loss: 2.2236 - accuracy: 0.0541
Epoch 2/100
3/3 [==============================] - 0s 8ms/step - loss: 2.2150 - accuracy: 0.1351
Epoch 3/100
3/3 [==============================] - 0s 6ms/step - loss: 2.1655 - accuracy: 0.1622
Epoch 4/100
3/3 [==============================] - 0s 6ms/step - loss: 2.1962 - accuracy: 0.1622
Epoch 5/100
3/3 [==============================] - 0s 6ms/step - loss: 2.2062 - accuracy: 0.0811
Epoch 6/100
3/3 [==============================] - 0s 7ms/step - loss: 2.1571 - accuracy: 0.2703
Epoch 7/100
3/3 [==============================] - 0s 6ms/step - loss: 2.2044 - accuracy: 0.2432
Epoch 8/100
3/3 [==============================] - 0s 6ms/step - loss: 2.1819 - accuracy: 0.1622
Epoch 9/100
3/3 [==============================] - 0s 7ms/step - loss: 2.1646 - accuracy: 0.2432
Epoch 10/100
3/3 [==============================] - 0s 8ms/step - loss: 2.1604 - accuracy: 0.1622
Epoch 11/100
3/3 [===========

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.